In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import joblib
import random
import gc

2025-07-03 14:45:40.548649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [3]:
df = pd.read_hdf("./dataset.h5", key="df")
df

,Cens_C,Cens_Db,Cens_D,Cens_Eb,Cens_E,Cens_F,Cens_Gb,Cens_G,Cens_Ab,Cens_A,Cens_Bb,Cens_B,chord
0,0.452348,0.229748,0.361109,0.384247,0.289957,0.505736,0.160134,0.192324,0.237007,0.020101,0.073743,0.007858,FMin7
1,0.453124,0.230152,0.356336,0.383752,0.279558,0.509347,0.157981,0.191034,0.246102,0.018734,0.086234,0.008667,FMin7
2,0.452593,0.231150,0.351795,0.383554,0.269870,0.512432,0.156105,0.190244,0.254603,0.017387,0.098308,0.009395,FMin7
3,0.450910,0.232791,0.347583,0.383560,0.260952,0.514989,0.154537,0.189900,0.262471,0.016060,0.109787,0.010040,FMin7
4,0.448296,0.235113,0.343783,0.383615,0.252767,0.517028,0.153299,0.189957,0.269682,0.014843,0.120621,0.010685,FMin7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.468113,0.251520,0.445619,0.172908,0.268400,0.162083,0.330289,0.069028,0.260254,0.335813,0.023730,0.309683,D7
1194,0.455816,0.267095,0.427637,0.185984,0.272747,0.171784,0.333622,0.071528,0.293414,0.321448,0.024044,0.305137,D7
1195,0.440661,0.282623,0.407051,0.199023,0.276647,0.181347,0.336395,0.074615,0.328778,0.305893,0.024366,0.300389,D7
1196,0.422635,0.297590,0.384021,0.211749,0.279753,0.190376,0.338419,0.078126,0.365778,0.289019,0.024682,0.295702,D7


In [4]:
print("Chords")
print(df["chord"].value_counts())

Chords
chord
CMin7     11630
AbMin7    11611
GbMin7    11600
DbMin7    11514
FMin7     11264
EMin7     11163
AMin7     11118
DMin7     11039
BbMin7    11026
EbMin7    10884
GMin7     10876
BMin7     10825
AMaj7      6860
EMaj7      6518
GbMaj7     6493
CMaj7      6406
AbMaj7     6379
FMaj7      6278
BbMaj7     6270
GMaj7      6249
DbMaj7     6240
DMaj7      6212
EbMaj7     6185
BMaj7      6135
G7         2692
B7         2536
Db7        2428
Bb7        2427
Ab7        2426
C7         2400
D7         2399
Eb7        2389
A7         2384
Gb7        2359
E7         2345
F7         2340
Name: count, dtype: int64


In [5]:
y = df["chord"]
X = df.drop(columns="chord")
encoder = sk.preprocessing.LabelEncoder()
y_encoded = encoder.fit_transform(y)

joblib.dump(encoder, "./encoder.xz")

['./encoder.xz']

In [6]:
del df
gc.collect()

20

# LSTM

In [7]:
SEQUENCE_LEN = 40 # 0.1 * 20.0 = 2 sec of sequence data

X_seq, y_encoded_seq = None, None
X_seq_list = []
y_encoded_seq_list = []
for i in range(len(X) - SEQUENCE_LEN + 1):
    X_seq_list.append(X.values[i : i + SEQUENCE_LEN, :])
    y_encoded_seq_list.append(y_encoded[i + SEQUENCE_LEN - 1])

X_seq, y_encoded_seq = np.array(X_seq_list), np.array(y_encoded_seq_list)

print("X sequence shape: ", X_seq.shape)
print("y sequence shape: ", y_encoded_seq.shape)

del X_seq_list
del y_encoded_seq_list
gc.collect()

X sequence shape:  (239861, 40, 12)
y sequence shape:  (239861,)


0

In [8]:
X_seq_flat = X_seq.reshape(X_seq.shape[0], -1)
_, unique_idx = np.unique(X_seq_flat, axis=0, return_index=True)
unique_idx = np.sort(unique_idx)

X_seq = X_seq[unique_idx]
y_encoded_seq = y_encoded_seq[unique_idx]

print("X deduplicate shape: ", X_seq.shape)
print("y deuplicate shape:  ", y_encoded_seq.shape)

del X_seq_flat
gc.collect()

X deduplicate shape:  (239861, 40, 12)
y deuplicate shape:   (239861,)


0

In [9]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = sk.model_selection.train_test_split(
    X_seq,
    y_encoded_seq,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded_seq,
)

print(f"X_train: {len(X_seq_train)}")
print(f"y_train: {len(y_seq_train)}")
print(f"X_test:  {len(X_seq_test)}")
print(f"y_test:  {len(y_seq_test)}")

del X_seq
del y_encoded_seq
gc.collect()

X_train: 191888
y_train: 191888
X_test:  47973
y_test:  47973


0

In [10]:
print("Total feature: ", X_seq_train.shape[2])
print("Total class:   ", len(encoder.classes_))

Total feature:  12
Total class:    36


In [11]:
class_weights = sk.utils.class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_seq_train),
    y=y_seq_train,
)
class_weight_dict = dict(enumerate(class_weights))

In [12]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LEN, X_seq_train.shape[2]))),
    tf.keras.layers.Dropout(0.25),
        
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax'),
])

model_lstm.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model_lstm.summary()

/usr/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True,
)

history = model_lstm.fit(
    X_seq_train,
    y_seq_train,
    epochs=500,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping],
    class_weight=class_weight_dict,
)

2025-07-03 14:45:46.528527: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 294739200 exceeds 10% of free system memory.


Epoch 1/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 119s 24ms/step - accuracy: 0.4327 - loss: 2.0500 - val_accuracy: 0.6681 - val_loss: 1.1665
Epoch 2/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 126s 26ms/step - accuracy: 0.6613 - loss: 1.1115 - val_accuracy: 0.7333 - val_loss: 0.9373
Epoch 3/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 130s 27ms/step - accuracy: 0.7166 - loss: 0.8843 - val_accuracy: 0.7636 - val_loss: 0.8097
Epoch 4/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 128s 27ms/step - accuracy: 0.7505 - loss: 0.7636 - val_accuracy: 0.8001 - val_loss: 0.6785
Epoch 5/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 128s 27ms/step - accuracy: 0.7890 - loss: 0.6250 - val_accuracy: 0.8309 - val_loss: 0.5738
Epoch 6/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 129s 27ms/step - accuracy: 0.8109 - loss: 0.5370 - val_accuracy: 0.8408 - val_loss: 0.5262
Epoch 7/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 131s 27ms/step - accuracy: 0.8294 - loss: 0.4809 - val_accuracy: 0.8547 - val_loss: 0.4826
Epoch 8/500
4798/4798 ━━━━━━━━━━━━━━━━━━━━ 134s 28ms/step - ac

In [14]:
model_lstm.evaluate(X_seq_test, y_seq_test)
y_pred = np.argmax(model_lstm.predict(X_seq_test), axis=1)
print(
  sk.metrics.classification_report(y_seq_test, y_pred, target_names=encoder.classes_)
)

1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9465 - loss: 0.1673
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step
              precision    recall  f1-score   support

          A7       0.88      0.96      0.92       477
       AMaj7       0.94      0.96      0.95      1372
       AMin7       0.95      0.96      0.95      2224
         Ab7       0.89      0.97      0.93       485
      AbMaj7       0.96      0.95      0.95      1276
      AbMin7       0.96      0.93      0.95      2322
          B7       0.90      0.97      0.93       507
       BMaj7       0.94      0.95      0.95      1227
       BMin7       0.96      0.94      0.95      2165
         Bb7       0.92      0.97      0.94       485
      BbMaj7       0.94      0.96      0.95      1254
      BbMin7       0.95      0.95      0.95      2205
          C7       0.88      0.97      0.93       480
       CMaj7       0.92      0.97      0.95      1281
       CMin7       0.97      0.93      0.95      2326
          D7 

In [17]:
model_lstm.save("model_lstm_cens_seq40.keras")
pd.DataFrame(history.history).to_csv("model_lstm_cens_seq40_history.csv", index=False)